<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_function_call_currency_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Websockets: streaming with websockets

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install `pyautogen`:
```bash
pip install pyautogen
```

In [5]:
# %pip install "pyautogen>=0.2.3"

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.

In [6]:
from tempfile import TemporaryDirectory
from websockets.sync.client import connect as ws_connect
from autogen.io.websockets import IOWebsockets

import autogen
from autogen.cache import Cache

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-3.5-turbo", "gpt-3.5-turbo-16k"],
    },
)

It first looks for environment variable "OAI_CONFIG_LIST" which needs to be a valid json string. If that variable is not found, it then looks for a json file named "OAI_CONFIG_LIST". It filters the configs by models (you can filter by other keys as well). Only the models with matching names are kept in the list based on the filter condition.

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your Azure OpenAI API key here>',
        'base_url': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2024-02-15-preview',
    },
]
```

You can set the value of config_list in any way you prefer. Please refer to this [notebook](https://github.com/microsoft/autogen/blob/main/website/docs/llm_configuration.ipynb) for full code examples of the different methods.

## Defining `on_connect` function


In [7]:
def on_connect(iostream: IOWebsockets) -> None:
    try:
        print(f" - on_connect(): Connected to client using IOWebsockets {iostream}", flush=True)

        print(" - on_connect(): Receiving message from client.", flush=True)

        initial_msg = iostream.input()

        llm_config = {
            "config_list": config_list,
            "stream": True,
        }

        agent = autogen.ConversableAgent(
            name="chatbot",
            system_message="Complete a task given to you and reply TERMINATE when the task is done.",
            llm_config=llm_config,
            iostream=iostream,
        )

        # create a UserProxyAgent instance named "user_proxy"
        user_proxy = autogen.UserProxyAgent(
            name="user_proxy",
            system_message="A proxy for the user.",
            is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
            human_input_mode="NEVER",
            max_consecutive_auto_reply=10,
            code_execution_config=False,
            iostream=iostream,
        )

        # we will use a temporary directory as the cache path root to ensure fresh completion each time
        with TemporaryDirectory() as cache_path_root:
            with Cache.disk(cache_path_root=cache_path_root) as cache:
                print(
                    f" - on_connect(): Initiating chat with agent {agent} using message '{initial_msg}'",
                    flush=True,
                )
                user_proxy.initiate_chat(  # noqa: F704
                    agent,
                    message=initial_msg,
                    cache=cache,
                )
    except Exception as e:
        print(f" - on_connect(): Exception occurred: {e}", flush=True)
        raise

## Run websockets server in a thread and connect to it with websockets client

In [8]:
with IOWebsockets.run_server_in_thread(on_connect=on_connect, port=8765) as uri:
    print(f" - test_setup() with websocket server running on {uri}.", flush=True)

    with ws_connect(uri) as websocket:
        print(f" - Connected to server on {uri}", flush=True)

        print(" - Sending message to server.", flush=True)
        # websocket.send("2+2=?")
        websocket.send("Please write a poem about spring in a city of your choice.")

        while True:
            message = websocket.recv()
            message = message.decode("utf-8") if isinstance(message, bytes) else message
            # drop the newline character
            if message.endswith("\n"):
                message = message[:-1]

            print(message, end="", flush=True)

            if "TERMINATE" in message:
                print()
                print(" - Received TERMINATE message. Exiting.", flush=True)
                break

 - test_setup() with websocket server running on ws://127.0.0.1:8765.
 - _handler(): Client connected on <websockets.sync.server.ServerConnection object at 0x7fb1dea3f520>
 - on_connect(): Connected to client using IOWebsockets <autogen.io.websockets.IOWebsockets object at 0x7fb1dea3eda0>
 - Connected to server on ws://127.0.0.1:8765
 - on_connect(): Receiving message from client.
 - Sending message to server.
 - on_connect(): Initiating chat with agent <autogen.agentchat.conversable_agent.ConversableAgent object at 0x7fb1dea3ed10> using message 'Please write a poem about spring in a city of your choice.'
user_proxy (to chatbot):
Please write a poem about spring in a city of your choice.
--------------------------------------------------------------------------------
>>>>>>>> USING AUTO REPLY...In Paris, blooms a spring delight,
Along the Seine, the petals bright.
Café chairs spill to cobbled street,
As winter's chill makes its retreat.

The Eiffel stands, a silent guard,
Above the blo